In [34]:
#importing the necessary packages
import numpy as np
import pandas as pd
import pandas as pd, numpy, string
import re
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding, Input, Conv1D,GlobalMaxPooling1D, Dropout, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical


In [102]:
#Path of the dataset file
read_path = '/Users/Aditya/Documents/MIDAS/'

In [36]:
#Importing the csv file and converting it into pandas dataframe
data=pd.read_csv(read_path + 'flipkart_com-ecommerce_sample - flipkart_com-ecommerce_sample - flipkart_com-ecommerce_sample.csv')

data.head()
#Converting all elements of description column into string
data.description=data.description.astype(str)

In [37]:
data

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,7179d2f6c4ad50a17d014ca1d2815156,2015-12-01 10:15:43 +0000,http://www.flipkart.com/walldesign-small-vinyl...,WallDesign Small Vinyl Sticker,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",STIE7KFJAKSTDY9G,1500.0,730.0,"[""http://img6a.flixcart.com/image/wall-decorat...",False,Buy WallDesign Small Vinyl Sticker for Rs.730 ...,No rating available,No rating available,WallDesign,"{""product_specification""=>[{""key""=>""Number of ..."
19996,71ac419198359d37b8fe5e3fffdfee09,2015-12-01 10:15:43 +0000,http://www.flipkart.com/wallmantra-large-vinyl...,Wallmantra Large Vinyl Stickers Sticker,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",STIE9F5URNQGJCGH,1429.0,1143.0,"[""http://img6a.flixcart.com/image/sticker/z/g/...",False,Buy Wallmantra Large Vinyl Stickers Sticker fo...,No rating available,No rating available,Wallmantra,"{""product_specification""=>[{""key""=>""Number of ..."
19997,93e9d343837400ce0d7980874ece471c,2015-12-01 10:15:43 +0000,http://www.flipkart.com/elite-collection-mediu...,Elite Collection Medium Acrylic Sticker,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",STIE7VAYDKQZEBSD,1299.0,999.0,"[""http://img5a.flixcart.com/image/sticker/b/s/...",False,Buy Elite Collection Medium Acrylic Sticker fo...,No rating available,No rating available,Elite Collection,"{""product_specification""=>[{""key""=>""Number of ..."
19998,669e79b8fa5d9ae020841c0c97d5e935,2015-12-01 10:15:43 +0000,http://www.flipkart.com/elite-collection-mediu...,Elite Collection Medium Acrylic Sticker,"[""Baby Care >>

In [38]:
data.count()

uniq_id                    20000
crawl_timestamp            20000
product_url                20000
product_name               20000
product_category_tree      20000
pid                        20000
retail_price               19922
discounted_price           19922
image                      19997
is_FK_Advantage_product    20000
description                20000
product_rating             20000
overall_rating             20000
brand                      14136
product_specifications     19986
dtype: int64

In [39]:
data["product_category_tree"][0]

'["Clothing >> Women\'s Clothing >> Lingerie, Sleep & Swimwear >> Shorts >> Alisha Shorts >> Alisha Solid Women\'s Cycling Shorts"]'

In [40]:
type(data["product_category_tree"][0])

str

Other columns are not as useful in improving the accuracy of the model as these two columns

In [41]:
#Dropping all the columns of the dataset except 'product_category_tree' and 'description'
data.drop(data.columns.difference(['product_category_tree','description']), 1, inplace=True)

In [42]:
data.head()

,product_category_tree,description
0,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",Key Features of Alisha Solid Women's Cycling S...
1,"[""Furniture >> Living Room Furniture >> Sofa B...",FabHomeDecor Fabric Double Sofa Bed (Finish Co...
2,"[""Footwear >> Women's Footwear >> Ballerinas >...",Key Features of AW Bellies Sandals Wedges Heel...
3,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",Key Features of Alisha Solid Women's Cycling S...
4,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",Specifications of Sicons All Purpose Arnica Do...


Used Pandas.series.str.split to split the text where '>>' is present.

In [43]:
#Splitting the text to separate out the primary category
data['product_category_tree'] = data['product_category_tree'].str.split(pat=">>")

In [44]:
data['product_category_tree'][0]

['["Clothing ',
 " Women's Clothing ",
 ' Lingerie, Sleep & Swimwear ',
 ' Shorts ',
 ' Alisha Shorts ',
 ' Alisha Solid Women\'s Cycling Shorts"]']

In [45]:
type(data['product_category_tree'][0])

list

In [46]:
data['product_category_tree'][0][0].lstrip('["').rstrip(' ')

'Clothing'

In [47]:
type(data['product_category_tree'][0][1].strip(" "))

str

As the primary key is the first element in the list,the function selects the first element and removes the unnecessary part which results in clean text. 


In [48]:
#Iterating through the entire column and stripping the unncessary part
data['product_category_tree'] = data.apply(lambda row:  row['product_category_tree'][0].lstrip('["').rstrip(' '), axis=1)

In [49]:
data.head()

,product_category_tree,description
0,Clothing,Key Features of Alisha Solid Women's Cycling S...
1,Furniture,FabHomeDecor Fabric Double Sofa Bed (Finish Co...
2,Footwear,Key Features of AW Bellies Sandals Wedges Heel...
3,Clothing,Key Features of Alisha Solid Women's Cycling S...
4,Pet Supplies,Specifications of Sicons All Purpose Arnica Do...


In [50]:
data["product_category_tree"].value_counts()

Clothing                                                 6198
Jewellery                                                3531
Footwear                                                 1227
Mobiles & Accessories                                    1099
Automotive                                               1012
                                                         ... 
Dressberry Orange Synthetic  Clutch"]                       1
NEWGEN TECH EO-HS3303 179 Wired Headset (White)"]           1
SAY UV Sterilizer Solid Filter Cartridge (0, Pac..."]       1
ABEEZ Boys, Men, Girls (Black, Pack of 1)"]                 1
SMART DENIM Solid Women's White Denim Shorts"]              1
Name: product_category_tree, Length: 266, dtype: int64

In [51]:
data["product_category_tree"].unique()

array(['Clothing', 'Furniture', 'Footwear', 'Pet Supplies',
       'Eternal Gandhi Super Series Crystal Paper Weight..."]',
       'Pens & Stationery', 'Sports & Fitness',
       'Beauty and Personal Care',
       'Bengal Blooms Rose Artificial Plant  with Pot (3..."]',
       'Bags, Wallets & Belts', 'Home Decor & Festive Needs',
       'Automotive', 'Tools & Hardware',
       'Vishudh Printed Women\'s Straight Kurta"]',
       'Vishudh Printed Women\'s Anarkali Kurta"]',
       'BuildTrack PIR Wireless Motion Sensor - One Swit..."]',
       'Skayvon SUMMERSIBLE SINGLE PHASE PUMP CONTROLLER..."]',
       'MASARA Solid Women\'s Straight Kurta"]',
       'Skayvon SUBMERSIBBLE THREE PHASE PUMP CONTROLLER..."]',
       'Behringer Xenyx 502 Analog Sound Mixer"]',
       'Noor Embroidered Women\'s Straight Kurta"]',
       'Libas Printed Women\'s A-line Kurta"]',
       'Libas Printed Women\'s Anarkali Kurta"]', 'Home Furnishing',
       'Baby Care', 'Mobiles & Accessories', 'Food & Nutriti

Some categories are not clearly defined and so it was important to observe the data carefully and see which categories are clearly defined and important.

In [52]:

data['product_category_tree'].value_counts()[:28].sort_values(ascending=False)

Clothing                             6198
Jewellery                            3531
Footwear                             1227
Mobiles & Accessories                1099
Automotive                           1012
Home Decor & Festive Needs            929
Beauty and Personal Care              710
Home Furnishing                       700
Kitchen & Dining                      647
Computers                             578
Watches                               530
Baby Care                             483
Tools & Hardware                      391
Toys & School Supplies                330
Pens & Stationery                     313
Bags, Wallets & Belts                 265
Furniture                             180
Sports & Fitness                      166
Cameras & Accessories                  82
Home Improvement                       81
Health & Personal Care Appliances      43
Sunglasses                             35
Gaming                                 35
Pet Supplies                      

Removing the categories whose value counts are less than 100 because the dataset would have become unbalanced.The data is very less for those categories.

In [53]:

counts = data['product_category_tree'].value_counts()
#Removing the unncessary categories and storing the remaining categories in a new dataframe
new_data = data.loc[data['product_category_tree'].isin(counts.index[counts > 100])]

In [54]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19289 entries, 0 to 19999
Data columns (total 2 columns):
product_category_tree    19289 non-null object
description              19289 non-null object
dtypes: object(2)
memory usage: 452.1+ KB


In [55]:
new_data.head()

,product_category_tree,description
0,Clothing,Key Features of Alisha Solid Women's Cycling S...
1,Furniture,FabHomeDecor Fabric Double Sofa Bed (Finish Co...
2,Footwear,Key Features of AW Bellies Sandals Wedges Heel...
3,Clothing,Key Features of Alisha Solid Women's Cycling S...
6,Clothing,Key Features of Alisha Solid Women's Cycling S...


In [73]:
new_data["product_category_tree"].unique()

array([ 4,  7,  6, 13, 14,  3,  2,  8,  0, 15,  9,  1, 12, 17, 16, 10, 11,
        5])

In [90]:
#Converted the categorical variable 'product_category_tree' from string into integer.
new_data["product_category_tree"] = new_data["product_category_tree"].astype('category')
new_data["product_category_tree"] = new_data["product_category_tree"].cat.codes

/Users/aditya/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/aditya/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [76]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19289 entries, 0 to 19999
Data columns (total 2 columns):
product_category_tree    19289 non-null int8
description              19289 non-null object
dtypes: int8(1), object(1)
memory usage: 960.2+ KB


In [77]:
new_data['product_category_tree']

0        4
1        7
2        6
3        4
6        4
        ..
19995    1
19996    1
19997    1
19998    1
19999    1
Name: product_category_tree, Length: 19289, dtype: int8

In [78]:
new_data["product_category_tree"].unique()

array([ 4,  7,  6, 13, 14,  3,  2,  8,  0, 15,  9,  1, 12, 17, 16, 10, 11,
        5])

In [79]:
#Function for removing punctuations in the 'description' column of the dataset
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

new_data['description'] = new_data['description'].apply(lambda x: remove_punct(x))
new_data.head(10)

/Users/aditya/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,product_category_tree,description
0,4,Key Features of Alisha Solid Womens Cycling Sh...
1,7,FabHomeDecor Fabric Double Sofa Bed Finish Col...
2,6,Key Features of AW Bellies Sandals Wedges Heel...
3,4,Key Features of Alisha Solid Womens Cycling Sh...
6,4,Key Features of Alisha Solid Womens Cycling Sh...
7,7,FabHomeDecor Fabric Double Sofa Bed Finish Col...
8,6,Key Features of dilli bazaaar Bellies Corporat...
9,4,Key Features of Alisha Solid Womens Cycling Sh...
10,6,Key Features of Ladela Bellies Brand LADELA Co...
11,4,Key Features of Carrel Printed Womens Fabric S...


In [101]:

write_path = '/Users/Aditya/Documents/MIDAS/'

In [84]:
new_data.product_category_tree.value_counts()

4     6198
10    3531
6     1227
12    1099
0     1012
8      929
3      710
9      700
11     647
5      578
17     530
1      483
15     391
16     330
13     313
2      265
7      180
14     166
Name: product_category_tree, dtype: int64

In [85]:
num_classes = new_data.product_category_tree.nunique()
print(num_classes)

18


Used Keras tokenizer function which tokenizes and assigns a unique integer to each word.A neural network model needs integer encoded words as it's input and not strings.Each sentence is converted into sequences of integrs.As each sentence as unequal length,it is necessary to make all the lenghts equal by padding the shorter sequences.Maxlen is calulated which helps to provide the length of the largest sentence.

In [91]:
#maximum number of words from the resulting tokenized data vocabulary
num_words = 5000
# Tokenize our training data
tokenizer = Tokenizer(num_words=num_words, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                   lower=True,split=' ')
print(new_data['description'][0])
tokenizer.fit_on_texts(new_data['description'].values)

# Get our training data word index
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
# Encode training data sentences into sequences
X = tokenizer.texts_to_sequences(new_data['description'].values)
print(X[0])
#Get max training sequence length
maxlen = max([len(x) for x in X])
# Pad the training sequences
X = pad_sequences(X, maxlen=maxlen)

print(word_index)
print("Padded Sequences: ")
print(X)
print(X[0])

Key Features of Alisha Solid Womens Cycling Shorts Cotton Lycra Navy Red NavySpecifications of Alisha Solid Womens Cycling Shorts Shorts Details Number of Contents in Sales Package Pack of  Fabric Cotton Lycra Type Cycling Shorts General Details Pattern Solid Ideal For Womens Fabric Care Gentle Machine Wash in Lukewarm Water Do Not Bleach Additional Details Style Code ALTHTP In the Box  shorts
Found 23927 unique tokens.
[61, 31, 1, 4562, 48, 24, 1214, 309, 46, 651, 885, 138, 1, 4562, 48, 24, 1214, 309, 309, 35, 57, 1, 107, 6, 68, 62, 47, 1, 37, 46, 651, 39, 1214, 309, 43, 35, 109, 48, 60, 2, 24, 37, 132, 610, 302, 141, 6, 890, 178, 175, 92, 330, 130, 35, 77, 149, 6, 3, 45, 309]
{'of': 1, 'for': 2, 'the': 3, 'and': 4, 'only': 5, 'in': 6, 'rs': 7, 'to': 8, 'with': 9, 'a': 10, 'on': 11, 'buy': 12, 'flipkartcom': 13, 'online': 14, 'from': 15, 'your': 16, 'is': 17, 'at': 18, 'this': 19, 'products': 20, 'free': 21, 'day': 22, 'genuine': 23, 'womens': 24, 'delivery': 25, 'cash': 26, 'shipping

In [ ]:
maxlen = max([len(x) for x in X])
maxlen

In [ ]:
X.shape

In [87]:
y = new_data['product_category_tree']

In [88]:
y

0        4
1        7
2        6
3        4
6        4
        ..
19995    1
19996    1
19997    1
19998    1
19999    1
Name: product_category_tree, Length: 19289, dtype: int8

In [92]:
#Apply one-hot encoding to the integer values of the column
Y = to_categorical(y, num_classes=num_classes)

In [93]:
Y[0]

array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.], dtype=float32)

In [94]:
y.shape

(19289,)

In [74]:
#Split the dataset into train and test data.
X_t, X_test, y_t, y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)
print(X_t.shape,y_t.shape)
print(X_test.shape,y_test.shape)

(15431, 701) (15431, 18)
(3858, 701) (3858, 18)


As there is no test data,the given dataset was splitted into three parts-Training set, Validation set, Test set.The training set is the majority of the actual dataset.


In [75]:
#Split the training data into actual training data and validation data
X_train, X_valid, y_train, y_valid = train_test_split(X_t,y_t, test_size = 0.2, random_state = 42)
print(X_train.shape,y_train.shape)
print(X_valid.shape,y_valid.shape)

(12344, 701) (12344, 18)
(3087, 701) (3087, 18)


In [95]:
# Set Hyper parameters
embed_dim = 100
batch_size = 64
learn_rate = 0.001 * 5
epochs = 25
print(num_classes)
print(maxlen)
print(num_words)

18
701
5000


In order to save the best model weights,model_checkpoint is used.The file path defined is where the weights are being saved.It is useful because in future we can just load the model with these weights.

In [121]:
#Set model_checkpoint callback to store the best weights of the model
checkpoint_filepath = write_path + "/tmp/checkpoint"
#Store the weights in defined path
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
Earlystopping callback is used to stop the training process when the required value is not the way which is desired.Here,the training stops when validation loss goes on increasing for 5 epochs.

In [132]:
#Define early_stopping callback
early_stopping = EarlyStopping(
    monitor='val_loss', min_delta=0, patience=5, verbose=1,
    mode='auto', baseline=None, restore_best_weights=True
)

First an embedding layer is defined which helps to vectorize each word in a sentence.It creates an embedding for each word in the vector space.Then GlobalMaxPooling is used to reduce the dimensions and parameters.Then the 2d matrix is converted into 1d with the dense layer and a dropout layer is attached.Then another dense layer is attached.As it's multiclass classification,loss chosen is categorical_crossentropy and activation function chosen is softmax.Adam optimizer is used.


In [96]:
#Define model
def get_model(learn_rate):
    model = Sequential()

    model.add(Embedding(num_words,
                        embed_dim,
                        input_length = maxlen))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(64))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Dense(32))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    optimizer = Adam(lr=learn_rate)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.summary()
    return model

In [134]:
get_model(learn_rate)

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_20 (Embedding)     (None, 701, 100)          500000    
_________________________________________________________________
global_max_pooling1d_11 (Glo (None, 100)               0         
_________________________________________________________________
dense_50 (Dense)             (None, 64)                6464      
_________________________________________________________________
dropout_32 (Dropout)         (None, 64)                0         
_________________________________________________________________
activation_50 (Activation)   (None, 64)                0         
_________________________________________________________________
dense_51 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_33 (Dropout)         (None, 32)              

In [97]:
#Calling function get_model and storing it in variable model
model = get_model(learn_rate)
#Fitting the model on training data
model.fit(X_train, 
          y_train, 
          validation_data=(X_valid,y_valid), 
          batch_size = batch_size, 
          epochs = epochs,
        callbacks=[early_stopping, model_checkpoint_callback])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 701, 100)          500000    
_________________________________________________________________
global_max_pooling1d (Global (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                6464      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
activation (Activation)      (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0

NameError: name 'X_train' is not defined

In [131]:
#Predicting the categories
ypred = model.predict(X_test)

In [98]:
ypred

NameError: name 'ypred' is not defined

In [137]:
ypred[0]

array([2.4272826e-04, 1.6730704e-05, 1.4170608e-03, 3.9962947e-04,
       5.6223519e-07, 7.7434123e-01, 7.3526517e-06, 1.9812508e-06,
       1.6757812e-04, 5.8788128e-08, 6.7914128e-02, 6.7954786e-02,
       1.3929597e-04, 5.9132524e-02, 3.1217377e-04, 1.5307133e-04,
       2.7646028e-02, 1.5314901e-04], dtype=float32)

Using argmax function to get the node with maximum probaility.

In [99]:

ypred_int = np.argmax(ypred, axis=-1)

NameError: name 'ypred' is not defined

In [139]:
ypred_int

array([ 5,  4,  2, ...,  6, 12,  4])

In [140]:
y_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [141]:
y_test[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.], dtype=float32)

In [142]:
#Taking max to again convert one-hot encoding into integer encoding
ytest_int = np.argmax(y_test, axis=-1)

In [143]:
ytest_int

array([ 5,  4,  4, ...,  6, 12,  4])

In [144]:
#Evaluate model
model.evaluate(X_test, y_test)

3858/3858 [==============================] - 0s 109us/sample - loss: 0.1512 - accuracy: 0.9676


[0.15122914285618816, 0.9675998]

In [ ]:
Comparing elements ypred_int and ytest_int to get the count of how many classes are matching with the original test data

In [145]:
count = 0
for i,val in enumerate(ypred_int):
    gt = ytest_int[i]
    if int(gt) == int(val):
        count +=1

In [146]:
count

3720

The correct category has been predicted 3720 times out of 3858.

In [147]:
len(ypred_int)

3858

Model accuracy is calculated 

In [148]:
model_accuracy = count/len(ypred_int)
model_accuracy

0.9642301710730948

But the problem is that the dataset is unbalanced , so some categories which are very few in the dataset might be incorrectly predicted.Even though the average accuracy is around 96%,that might be because of the correct prediction of categories whose value_counts are very large.It is not good that the data some categories are incorrectly predicted by it is not reflecting in the final accuracy.So it is important to get accuracy for each category.

In [149]:
#Find the number of individual categories
#and store the values in dictionary
ground_truth_dict = {}
for val in ytest_int:
    # type casting floats to int
    val = int(val)
    #Check if value is present in ground_truth_dict
    if val not in ground_truth_dict:
        ground_truth_dict[val] = 0
    ground_truth_dict[val] += 1

In [150]:
ground_truth_dict

{5: 122,
 4: 1191,
 10: 710,
 11: 131,
 6: 258,
 9: 131,
 12: 228,
 0: 224,
 1: 95,
 17: 109,
 16: 62,
 15: 77,
 7: 34,
 8: 185,
 2: 50,
 3: 137,
 13: 77,
 14: 37}

The number of times each value is correctly predicted is found out and the corresponding values are stored in a dictionary

In [151]:
y_pred_dict = {}
#Find the number of times each value is correctly predicted
#and store the values in dictionary
for i in range(len(ypred_int)):
    gt = int(ytest_int[i])
    val = int(ypred_int[i])
    #Check if both values are same
    if gt == val:
        if val not in y_pred_dict:
            y_pred_dict[val] = 0
        y_pred_dict[val] += 1

In [152]:
y_pred_dict

{5: 118,
 4: 1175,
 10: 705,
 11: 123,
 6: 256,
 9: 125,
 12: 219,
 0: 222,
 1: 81,
 17: 108,
 16: 57,
 15: 73,
 8: 179,
 2: 44,
 3: 133,
 13: 45,
 14: 25,
 7: 32}

The accuracies are found out for each category and it's visible that the categories which appear very few times in the dataset have less accuracy than categories which appear the most beacuse the data is unbalanced.That's why this technique was used to find individual categories accuracy

In [153]:
#  find the accuracy of individual class labels 
# and store the accuracies in dict
acc_dict = {}
# Iterate on key-value pair of ground truth dict
for key, val in ground_truth_dict.items():
    acc_dict[key] = 0.
    # Check if key is present in ground truth dict
    if key in y_pred_dict:
        # Get the ratio of ypred value counts to ground 
        # truth value counts for every key
        val2 = y_pred_dict[key]
        ratio = val2/val
        # Save the accuracy retio
        acc_dict[key] = ratio

In [154]:
acc_dict

{5: 0.9672131147540983,
 4: 0.9865659109991604,
 10: 0.9929577464788732,
 11: 0.9389312977099237,
 6: 0.9922480620155039,
 9: 0.9541984732824428,
 12: 0.9605263157894737,
 0: 0.9910714285714286,
 1: 0.8526315789473684,
 17: 0.9908256880733946,
 16: 0.9193548387096774,
 15: 0.948051948051948,
 7: 0.9411764705882353,
 8: 0.9675675675675676,
 2: 0.88,
 3: 0.9708029197080292,
 13: 0.5844155844155844,
 14: 0.6756756756756757}

In [155]:
new_model = get_model(learn_rate)

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_22 (Embedding)     (None, 701, 100)          500000    
_________________________________________________________________
global_max_pooling1d_13 (Glo (None, 100)               0         
_________________________________________________________________
dense_56 (Dense)             (None, 64)                6464      
_________________________________________________________________
dropout_36 (Dropout)         (None, 64)                0         
_________________________________________________________________
activation_56 (Activation)   (None, 64)                0         
_________________________________________________________________
dense_57 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_37 (Dropout)         (None, 32)              

In [156]:
new_model.load_weights(checkpoint_filepath)

In [158]:
new_model.evaluate(X_test, y_test)

3858/3858 [==============================] - 0s 108us/sample - loss: 0.1528 - accuracy: 0.9705


[0.15279783849997186, 0.970451]